This notebook has all functions.
Methods :

Method 1 : Threshold-->Filter-->Erosion-->Dilation
Method 2 : Filter-->Threshold-->Erosion-->Dilation

In [1]:
# to save labelled images
BASE_DIR="/Users/Trupti/01-LIDo/02-VijiProject/ImageAnalysis/" 


In [2]:
def save_img(folder_path,img_name,labelled_name):
    LABELLED_IMG_DIR = BASE_DIR + "AnalysisMethods/AnalysisResults/XMovie/set2-pp1/labelled_images/"
    directory=folder_path.split('/')[-1].split('.')[0] # to create a folder per experiment to save images
    path = LABELLED_IMG_DIR  + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    plt.imsave((path + '/' +'labelled'+'_'+ labelled_name +'.png'),img_name,dpi=300)
    

In [ ]:
def outliers(df):
    Q1 = df['YFP_pp1@kt/YFP_pp1@cyto'].quantile(0.25)
    Q3 = df['YFP_pp1@kt/YFP_pp1@cyto'].quantile(0.75)
    IQR = Q3 - Q1
    df_out= df[~((df['YFP_pp1@kt/YFP_pp1@cyto'] < (Q1 - 1.5 * IQR)) |(df['YFP_pp1@kt/YFP_pp1@cyto'] > (Q3 + 1.5 * IQR)))]
    return(df_out)

In [3]:
def cytoplasm_signal(img):
  '''
  This function takes an 8bit image,calculates the pixel value for all 4 corners in a 5x5 window 
  and returns its mean.
  '''
  col,row=img.shape
  topLeft=img[0:10, 0:10].flatten() 
  topRight=img[col-10:col,0:10].flatten()
  bottomLeft=img[0:10,col-10:col].flatten() 
  bottomRight=img[col-10:col,col-10:col].flatten()

  mean_array=np.concatenate([topLeft,topRight,bottomLeft,bottomRight])
  mean=np.mean(mean_array)
  
  return(mean)

In [6]:
def nuf2(nuf2img,nuf2im,propList):
    '''
    This function calculates the intensity of Nuf2-GFP on KT
    Image Used : nuf2 Image
    
    '''
    #create a nuf2 for the particles
    #Apply threshold
    threshold = filters.threshold_yen(nuf2img)
    #Generate thresholded image
    threshold_image = nuf2img > threshold
    # Apply prewitt filter to threshold image
    prewitt_im= filters.prewitt(threshold_image)
        #Apply erosion to the filtered image followed by dilation to the eroded image

    erosion_im=morphology.binary_erosion(prewitt_im, selem=None, out=None)
    dilation_im=morphology.binary_dilation(erosion_im, selem=None, out=None)

    labelled_nuf2,num_labels=ndi.label(dilation_im)


    all_props=measure.regionprops_table(labelled_nuf2, intensity_image=nuf2im, properties=propList) # intensity image is 16 bit red pp1 image
    df_nuf2= pd.DataFrame(all_props)
   
    return(df_nuf2,dilation_im,labelled_nuf2)

In [2]:
def pp1(particle_nuf2,pp1_img,pp1_im,propList):
    '''
    This function calculates the pp1 intensity on KT
   
    '''
    # find the co-ordinates of the rest of the image without the particles
    # dilation_im is the binary image with particles 1 and bg 0

    rest_of_img_idx=np.where(particle_nuf2==0)
    # copy pp1 image into another for further operations
    pp1_pp1_img= pp1_img.copy()
    pp1_pp1_img[rest_of_img_idx]=0
    # create a mask for the pp1 after removing the background. We are interested only in the 
    # particles locations
    threshold=filters.threshold_triangle(pp1_pp1_img)
    pp1_mask =pp1_pp1_img> threshold # main cell stays black
    # find the intensity of pp1 at the nuf2 signal locations without the cytoplasm
    labelled_pp1,num_labels_pp1=ndi.label(pp1_mask)
    all_props=measure.regionprops_table(labelled_pp1, intensity_image=pp1_im, properties=propList)#['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red pp1 image
    df_pp1= pd.DataFrame(all_props)
    return(df_pp1)

In [3]:
def nuf2_cyto_fun_for_pp1(particle_nuf2,nuf2img,nuf2im):
    '''
    Nuf2-Cyto : Nuf2-GFP cytoplasmic intensity
    Intensity of Nuf2 signal cell without the particles (nuf2 image)
    '''
    threshold=filters.threshold_mean(nuf2img)
    nuf2_cyto =nuf2img > threshold
    nuf2_cyto = morphology.remove_small_objects(nuf2_cyto)
    nuf2_cyto = morphology.remove_small_holes(nuf2_cyto)
    # black idx we need to remove the particles. hence select the particles idx 
    idx = np.where(particle_nuf2 != 0)
    nuf2_cyto[idx]=0 
    labelled_nuf2_cyto,num_labels_nuf2=ndi.label(nuf2_cyto)
    all_props=measure.regionprops_table(labelled_nuf2_cyto, intensity_image=nuf2im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red pp1 image
    df_nuf2_cyto= pd.DataFrame(all_props)
    
    try:
        nuf2_cyto_value=df_nuf2_cyto[df_nuf2_cyto['area']==df_nuf2_cyto['area'].max()]['mean_intensity'].values[0]
        
        return(nuf2_cyto_value)
    except:
        return(np.nan)

In [12]:
def nuf2_cyto_fun_for_astrin(particle_nuf2,nuf2img,nuf2im):
    '''
    Nuf2-Cyto : Nuf2-GFP cytoplasmic intensity
    Intensity of Nuf2 signal cell without the particles (nuf2 image)
    '''
    threshold=filters.threshold_mean(nuf2img)
    nuf2_cyto =nuf2img > threshold
    nuf2_cyto = morphology.remove_small_objects(nuf2_cyto)
    nuf2_cyto = morphology.remove_small_holes(nuf2_cyto)
    # black idx we need to remove the particles. hence select the particles idx 
    idx = np.where(particle_nuf2 != 0)
    nuf2_cyto[idx]=0 
    labelled_nuf2_cyto,num_labels_nuf2=ndi.label(nuf2_cyto)
    all_props=measure.regionprops_table(labelled_nuf2_cyto, intensity_image=nuf2im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red pp1 image
    df_nuf2_cyto= pd.DataFrame(all_props)
    
    try:
        nuf2_cyto_value=df_nuf2_cyto[df_nuf2_cyto['area']==df_nuf2_cyto['area'].max()]['mean_intensity'].values[0]
        return(nuf2_cyto_value)
    except:
        return(np.nan)

In [13]:
def pp1_cyto_fun(particle_nuf2,pp1_img,pp1_im):
    
    '''
    PP1 Cyto : YFP-PP1 cytoplasmic intensity
    Intensity of YFP-PP1 signal cell without the particles (pp1 image)
    
    '''
    # mask for the cell
    threshold=filters.threshold_mean(pp1_img)

    pp1_cyto =pp1_img >threshold
    pp1_cyto = morphology.remove_small_objects(pp1_cyto)
    pp1_cyto = morphology.remove_small_holes(pp1_cyto)
    
     #black idx # remove KTs

    kt_idx = np.where(particle_nuf2 != 0)
    pp1_cyto[kt_idx]=0 
                                                     
    labelled_pp1_cyto,num_labels_pp1_cyto=ndi.label(pp1_cyto)
    all_props=measure.regionprops_table(labelled_pp1_cyto, intensity_image=pp1_im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red pp1 image
    df_pp1_cyto= pd.DataFrame(all_props)

    try:
        pp1_cyto_value=df_pp1_cyto[df_pp1_cyto['area']==df_pp1_cyto['area'].max()]['mean_intensity'].values[0]
        return(pp1_cyto_value)
    except:
        return(np.nan)
    
     

In [14]:
def astrin_cyto_fun(particle_nuf2,astrin_img,astrin_im): #
    
    '''
    Astrin Cyto : mKate2-Astrin cytoplasmic intensity
    Intensity of mKate2-Astrin signal cell without the particles (astrin image)
    
    '''
    # mask for the cell
    threshold=filters.threshold_mean(astrin_img)

    astrin_cyto =astrin_img >threshold
    astrin_cyto = morphology.remove_small_objects(astrin_cyto)
    astrin_cyto = morphology.remove_small_holes(astrin_cyto)
    # find microtubule

    triangle_threshold=filters.threshold_triangle(astrin_img)
    microtubule=astrin_img < triangle_threshold

    # remove microtubule 
    mt_idx = np.where(microtubule== 0)
    astrin_cyto[mt_idx]=0
    #black idx # remove KTs

    kt_idx = np.where(particle_nuf2 != 0)
    astrin_cyto[kt_idx]=0 
                                                     
    labelled_astrin_cyto,num_labels_astrin_cyto=ndi.label(astrin_cyto)
    all_props=measure.regionprops_table(labelled_astrin_cyto, intensity_image=astrin_im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red pp1 image
    df_astrin_cyto= pd.DataFrame(all_props)

    try:
        astrin_cyto_value=df_astrin_cyto[df_astrin_cyto['area']==df_astrin_cyto['area'].max()]['mean_intensity'].values[0]
        return(astrin_cyto_value)
    except:
        return(np.nan)
    

In [15]:
def method_pp1(folder_path):
    '''
      This function takes the folder path of tif images and performs following steps.
      1. Reads the image from the path
      2. Converts the 16bit image to 8 bit
      3. Prewitt Filter-->Yen Threshold-->Erosion-->dilation
  
      For mean intensity calculation, the background noise needs to be filtered from the intensity image. 

    '''
    propList = ['label','area','mean_intensity'] 
    df_nuf2_final = pd.DataFrame(columns=propList)
    df_pp1_final= pd.DataFrame(columns=propList)
    
    # set path for images
    nuf2_cfp_path = os.path.join(folder_path,"nuf2-cfp","*.tif") #C1 nuf2 signal
    yfp_pp1_path = os.path.join(folder_path,"yfp-pp1","*.tif") #C0 pp1 signal
    # create nuf2-signal image array
    pp1_image_list=[]
    for file in natsorted(glob.glob(yfp_pp1_path)):
        pp1_image_list.append(file)
     
    k=0
    for file in natsorted(glob.glob(nuf2_cfp_path)):
        nuf2_im= io.imread(file)  # This is to measure and label the particles
        #Convert an (ImageJ) TIFF to an 8 bit numpy array
        nuf2_img= (nuf2_im / np.amax(nuf2_im) * 255).astype(np.uint8)
        pp1_im = io.imread(pp1_image_list[k])
        pp1_img= (pp1_im / np.amax(pp1_im) * 255).astype(np.uint8)
        
        # get nuf2 signal
        df_nuf2,particle_nuf2,labelled_nuf2=nuf2(nuf2_img,nuf2_im,propList)
        nuf2_fname=re.sub(r'^.+/([^/]+)$', r'\1',file)
        #df_nuf2.insert(0,'nuf2_fname',nuf_fname)
        df_nuf2['label']=str(k) +"_"+ df_nuf2['label'].astype(str)
        df_nuf2['fname']=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
       
        # get pp1 signal
        df_pp1=pp1(particle_nuf2,pp1_img,pp1_im,propList)        
        
        df_pp1['fname']=re.sub(r'^.+/([^/]+)$', r'\1',pp1_image_list[k])
        df_pp1['label']=str(k) +"_"+ df_pp1['label'].astype(str)
        
        # cytoplasm intensity of nuf2 image
        nuf2_cyto_value=nuf2_cyto_fun_for_pp1(particle_nuf2,nuf2_img,nuf2_im)
        
        #cytoplasm intensity for pp1 image
        pp1_cyto_value=pp1_cyto_fun(particle_nuf2,pp1_img,pp1_im)
        # nuf2 bg outside the cell
        nuf2_bg=cytoplasm_signal(nuf2_im)
        # pp1 bg outside the cell
        pp1_bg=cytoplasm_signal(pp1_im)
        
        df_nuf2_final=pd.concat([df_nuf2_final,df_nuf2],ignore_index=True)        
        df_pp1_final=pd.concat([df_pp1_final,df_pp1],ignore_index=True)
       
        
        # #SAVE THE IMAGES : uncomment to save the images
        #labelled_name=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
        #save_img(folder_path,labelled_nuf2,labelled_name)
       
       
        
        k+=1
  
    return(df_nuf2_final,df_pp1_final,nuf2_cyto_value,pp1_cyto_value,nuf2_bg,pp1_bg)
              

In [ ]:
def method_astrin(folder_path):
    '''
      This function takes the folder path of tif images and performs following steps.
      1. Reads the image from the path
      2. Converts the 16bit image to 8 bit
      3. Prewitt Filter-->Yen Threshold-->Erosion-->dilation
  
      For mean intensity calculation, the background noise needs to be filtered from the intensity image. 

    '''
    propList = ['label','area', 'mean_intensity'] 
    df_nuf2_final = pd.DataFrame(columns=propList)
    df_pp1_final= pd.DataFrame(columns=propList)
    
    # set path for images
    astrin_chpath = os.path.join(folder_path,"mKate2-astrin","*.tif") #C1 astrin
    nuf2_chpath = os.path.join(folder_path,"nuf2-GFP","*.tif") #C0 nuf2
    # create astrin image array
    astrin_image_list=[]
    for file in natsorted(glob.glob(astrin_chpath)):
        astrin_image_list.append(file)
     
    k=0
    for file in natsorted(glob.glob(nuf2_chpath)):
        nuf2_im= io.imread(file)  # This is to measure and label the particles
        #Convert an (ImageJ) TIFF to an 8 bit numpy array
        nuf2_img= (nuf2_im / np.amax(nuf2_im) * 255).astype(np.uint8)
        astrin_im = io.imread(astrin_image_list[k])
        astrin_img= (astrin_im / np.amax(astrin_im) * 255).astype(np.uint8)
        
        # get nuf2 signal
        df_nuf2,particle_nuf2,labelled_nuf2=nuf2(nuf2_img,nuf2_im,propList)
        nuf2_fname=re.sub(r'^.+/([^/]+)$', r'\1',file)
        #df_nuf2.insert(0,'nuf2_fname',nuf_fname)
        df_nuf2['label']=str(k) +"_"+ df_nuf2['label'].astype(str)
        df_nuf2['fname']=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
       
        # get astrin signal
        df_astrin=astrin(particle_nuf2,pp1_img,astrin_im,propList)        
        as_fname=re.sub(r'^.+/([^/]+)$', r'\1',astrin_image_list[k])
        #df_astrin.insert(0,'astrin_fname',as_fname)
        df_astrin['fname']=re.sub(r'^.+/([^/]+)$', r'\1',astrin_image_list[k])
        df_astrin['label']=str(k) +"_"+ df_astrin['label'].astype(str)
        
        # cytoplasm intensity of nuf2 image
        nuf2_cyto_value=nuf2_cyto_fun_for_astrin(particle_nuf2,nuf2_img,nuf2_im)
        #cytoplasm intensity for astrin image
        astrin_cyto_value=astrin_cyto_fun(particle_nuf2,astrin_img,astrin_im)
        # nuf2 bg outside the cell
        nuf2_bg=cytoplasm_signal(nuf2_im)
        # astrin bg outside the cell
        astrin_bg=cytoplasm_signal(astrin_im)
        
        df_nuf2_final=pd.concat([df_nuf2_final,df_nuf2],ignore_index=True)        
        df_astrin_final=pd.concat([df_astrin_final,df_astrin],ignore_index=True)
        
        
        # #SAVE THE IMAGES : uncomment to save the images
        #labelled_name=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
        #save_img(folder_path,labelled_nuf2,labelled_name)
       
       
        
        k+=1
  
    return(df_nuf2_final,df_astrin_final,nuf2_cyto_value,astrin_cyto_value,nuf2_bg,astrin_bg)
            